In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
tes = pd.read_csv('/content/drive/MyDrive/social-networks-project/top5_collaborative_filtering_results.csv')

In [ ]:
len(tes)

286521

In [ ]:
dat = pd.read_csv('/content/drive/MyDrive/social-networks-project/Colab_sized_data.csv').drop('Unnamed: 0',1).drop('Unnamed: 0.1',1)

ParserError: ignored

In [ ]:
dat

NameError: ignored

In [ ]:
# Scoring function takes hours to run if all 30,000 articles are considered
most_pop = dat['article_id'].value_counts().index.tolist()[0:100]

In [ ]:
dat_pop = dat[dat['article_id'].isin(most_pop)]

In [ ]:
size = len(dat_pop)

In [ ]:
split = int(2 * (size / 3))

In [ ]:
dat_train = dat_pop[:split]
dat_test = dat_pop[split:]

In [ ]:
print(len(dat_train),len(dat_test))

116665 58333


In [ ]:
dat_train = pd.read_csv('/content/drive/MyDrive/social-networks-project/training_test/train_revise.csv').drop('Unnamed: 0',1)
dat_test = pd.read_csv('/content/drive/MyDrive/social-networks-project/training_test/test_revise.csv').drop('Unnamed: 0',1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


In [ ]:
print(len(dat_train),len(dat_test))

1781011 445253


What still needs to be done:

1) ~~For each article: Convert Categorical features to numerical values to compute similarity~~

2) ~~For each unique customer id: Generate mean feature value for each feature for each item the customer purchased to create customer 'profile'~~

3) ~~Create function to determine top 10 articles most similar to the customer profile~~

4) ~~For each customer id: Use the function to get the top 10 reccomended articles~~

5) ~~Create Evaluation function that checks if they actually bought the reccomended product~~

6) ~~Compute accuracy of reccomendation and compare against naive top-10-most-popular reccomender.~~

## Feature Construction

In [ ]:
dat_train

,t_dat,customer_id,article_id,price,sales_channel_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,perceived_colour_value_id,perceived_colour_value_name,perceived_colour_master_id,perceived_colour_master_name,department_no,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,2019-03-15,6892ef2f7ba91c663d197347c966a67bb57e6d7e1d01ea...,562245046,0.033881,2,NaN,NaN,PRE-CREATE,NONE,38.0,f715c90246db2ff88835acfd0318a99f601249832d9246...,562245,Luna skinny RW,272,Trousers,Garment Lower body,1010016,Solid,9,Black,4,Dark,5,Black,1722,Trouser,A,Ladieswear,1,Ladieswear,15,Womens Everyday Collection,1009,Trousers,"5-pocket jeans in washed, superstretch denim w..."
1,2019-01-25,87f10cb562b7f8f8257ecb742f0bc9f35271410dedd312...,458543001,0.030492,2,NaN,NaN,ACTIVE,NONE,31.0,c14b8f01ef9ca8d6fa4ab0224a12093b5a95d45aed637e...,458543,Lazer Razer Push Up,298,Bikini top,Swimwear,1010016,Solid,9,Black,4,Dark,5,Black,4242,Swimwear,B,Lingeries/Tights,1,Ladieswear,60,"Womens Swimwear, beachwear",1018,Swimwear,Fully lined bikini top with laser-cut scallope...
2,2019-02-08,9aea5d7f5e0f66f52243fd0059cbb391036570c5afac6f...,470789001,0.015237,1,1.0,1.0,ACTIVE,Regularly,51.0,91fafd28134aff357df81cd12505ca71a668a13e783d00...,470789,OP Brazilian 2p Low (Acacia),286,Underwear bottom,Underwear,1010016,Solid,9,Black,4,Dark,5,Black,1339,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear",Microfibre Brazilian briefs with laser-cut edg...
3,2018-10-26,20d7b82fb2e501fc59b6dd9ea76840525364d31f6f39f9...,501820003,0.016932,2,1.0,1.0,ACTIVE,Regularly,20.0,eea61f54d3522b45c3e0faf146067e84a96bc20649f7bc...,501820,SIRPA,252,Sweater,Garment Upper body,1010010,Melange,43,Dark Red,4,Dark,18,Red,1647,Tops Knitwear,D,Divided,2,Divided,53,Divided Collection,1003,Knitwear,Jumper in a soft knit with a slightly wider ne...
4,2018-11-29,e161bc69e8b8d3d0547f2fa686cbfefd045e6572fd6504...,711521004,0.025407,1,NaN,NaN,ACTIVE,NONE,23.0,421e3aa8511664dceaa5713c43aa23aca2f30f4e877527...,711521,CSP Eva,252,Sweater,Garment Upper body,1010016,Solid,22,Yellow,2,Medium Dusty,8,Yellow,1636,Jersey fancy,A,Ladieswear,1,Ladieswear,15,Womens Everyday Collection,1005,Jersey Fancy,"Fine-knit top with a soft, brushed finish. Sta..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1781006,2019-04-05,74685c9e5a36038c5ba963c8648e0556be7ee03f04cc97...,590928019,0.030492,2,NaN,NaN,ACTIVE,NONE,52.0,55c79c7c82a35e2b38c88608d53ee24c46a11fcd56944b...,590928,New Girl Push Top,298,Bikini top,Swimwear,1010001,All over pattern,31,Light Orange,1,Dusty Light,3,Orange,4242,Swimwear,B,Lingeries/Tights,1,Ladieswear,60,"Womens Swimwear, beachwear",1018,Swimwear,"Lined, push-up triangle bikini top with moulde..."
1781007,2018-12-08,85626680b54a74316a1a9ade489ed90beb3bc62dd00036...,673799001,0.033881,2,1.0,1.0,ACTIVE,Regularly,51.0,a1e2485152b9cce5adbabf7860d4e902aa66ae04ab0a54...,673799,Bow paperwaist,272,Trousers,Garment Lower body,1010016,Solid,9,Black,4,Dark,5,Black,1722,Trouser,A,Ladieswear,1,Ladieswear,15,Womens Everyday Collection,1009,Trousers,Ankle-length trousers in woven fabric with a h...
1781008,2019-02-04,a1fa1a370e84becc5a9692ec1735088d2fd92acd26335b...,529008003,0.025407,2,1.0,1.0,ACTIVE,Regularly,29.0,312d4a1ee2f60d6e63898acdb5a44da82bc9b042f0f2de...,529008,Hazelnut Push Melbourne,306,Bra,Underwear,1010016,Solid,9,Black,4,Dark,5,Black,1338,Expressive Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Lace push-up bra with underwired, moulded, pad..."
1781009,2019-02-15,010123767f5ea492f77771f77e3efcdf8501d5d075c1b1...,590928011,0.026237,2,NaN,NaN,ACTIVE,NONE,34.0,f6a

In [ ]:
categories_to_separate = [dat_train.product_group_name, dat_train.perceived_colour_master_name, dat_train.index_group_name,dat_train.section_name]

In [ ]:
for c in categories_to_separate:
   dat_train = pd.concat([dat_train, pd.get_dummies(c)], axis='columns')

In [ ]:
articles = dat_train.drop(['t_dat','age','customer_id','sales_channel_id','FN','Active','club_member_status','fashion_news_frequency','postal_code',
                                           'product_code','prod_name','product_type_no','product_type_name',	'product_group_name','graphical_appearance_no',
                                           'graphical_appearance_name','colour_group_code','colour_group_name','perceived_colour_value_id','perceived_colour_value_name',
                                           'perceived_colour_master_id','perceived_colour_master_name','department_no','department_name','index_code','index_name',
                                           'index_group_no','index_group_name','section_no','section_name','garment_group_no','garment_group_name','detail_desc'],1).groupby('article_id').mean()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """


In [ ]:
# Article Features - COMPLETES STEP 1
articles

,price,Accessories,Garment Full body,Garment Lower body,Garment Upper body,Nightwear,Shoes,Socks & Tights,Swimwear,Underwear,Unknown,Beige,Black,Blue,Brown,Green,Grey,Khaki green,Lilac Purple,Mole,Orange,Pink,Red,Turquoise,Unknown,White,Yellow,undefined,Baby/Children,Divided,Ladieswear,Menswear,Sport,Denim Men,Divided Accessories,Divided Basics,Divided Collection,Divided Complements Other,Divided Selected,H&M+,Ladies Denim,Ladies H&M Sport,Mama,Men Suits & Tailoring,Men Underwear,Womens Big accessories,Womens Casual,Womens Everyday Basics,Womens Everyday Collection,Womens Jackets,Womens Lingerie,"Womens Nightwear, Socks & Tigh",Womens Premium,Womens Shoes,Womens Small accessories,"Womens Swimwear, beachwear",Womens Tailoring,Young Boy,Young Girl
article_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
108775015,0.008150,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
108775044,0.008129,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
111565001,0.007431,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
111586001,0.012220,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
111593001,0.012417,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
762762002,0.016381,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
766346001,0.024717,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
767775001,0.032479,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
customers = dat_train.drop(['t_dat','age','article_id','sales_channel_id','FN','Active','club_member_status','fashion_news_frequency','postal_code',
                                           'product_code','prod_name','product_type_no','product_type_name',	'product_group_name','graphical_appearance_no',
                                           'graphical_appearance_name','colour_group_code','colour_group_name','perceived_colour_value_id','perceived_colour_value_name',
                                           'perceived_colour_master_id','perceived_colour_master_name','department_no','department_name','index_code','index_name',
                                           'index_group_no','index_group_name','section_no','section_name','garment_group_no','garment_group_name','detail_desc'],1).groupby('customer_id').mean()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """


In [ ]:
# Average features of articles each customer purchased COMPLETES STEP 2
customers

,price,Accessories,Garment Full body,Garment Lower body,Garment Upper body,Nightwear,Shoes,Socks & Tights,Swimwear,Underwear,Unknown,Beige,Black,Blue,Brown,Green,Grey,Khaki green,Lilac Purple,Mole,Orange,Pink,Red,Turquoise,Unknown,White,Yellow,undefined,Baby/Children,Divided,Ladieswear,Menswear,Sport,Denim Men,Divided Accessories,Divided Basics,Divided Collection,Divided Complements Other,Divided Selected,H&M+,Ladies Denim,Ladies H&M Sport,Mama,Men Suits & Tailoring,Men Underwear,Womens Big accessories,Womens Casual,Womens Everyday Basics,Womens Everyday Collection,Womens Jackets,Womens Lingerie,"Womens Nightwear, Socks & Tigh",Womens Premium,Womens Shoes,Womens Small accessories,"Womens Swimwear, beachwear",Womens Tailoring,Young Boy,Young Girl
customer_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0000423b00ade91418cceaf3b26c6af3dd342b51fd051eec9c12fb36984420fa,0.021169,0.0,0.000000,0.000000,1.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.500000,0.500000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,1.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.500000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.500000,0.0,0.0
00007d2de826758b65a93dd24ce629ed66842531df6699338c5570910a014cc2,0.018345,0.0,0.083333,0.166667,0.500000,0.0,0.0,0.0,0.000000,0.250000,0.0,0.000000,0.500000,0.000000,0.083333,0.083333,0.000000,0.0,0.0,0.0,0.000000,0.166667,0.000000,0.0,0.0,0.166667,0.0,0.0,0.0,0.500000,0.500000,0.0,0.000000,0.0,0.0,0.166667,0.250000,0.0,0.083333,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.166667,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
00008469a21b50b3d147c97135e25b4201a8c58997f78782a0cc706645e14493,0.016932,0.0,0.000000,0.000000,1.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.500000,0.000000,0.000000,0.000000,0.000000,0.500000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,1.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.5,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.500000,0.0,0.0
00009d946eec3ea54add5ba56d5210ea898def4b46c68570cf0096d962cacc75,0.029837,0.0,0.000000,0.636364,0.363636,0.0,0.0,0.0,0.000000,0.000000,0.0,0.090909,0.545455,0.090909,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.090909,0.000000,0.000000,0.0,0.0,0.181818,0.0,0.0,0.0,0.181818,0.636364,0.0,0.181818,0.0,0.0,0.090909,0.090909,0.0,0.000000,0.0,0.000000,0.181818,0.0,0.0,0.0,0.0,0.0,0.0,0.636364,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
0000b2f1829e23b24feec422ef13df3ccedaedc85368e6664d04ca30b2f8daff,0.015237,0.0,0.000000,0.000000,1.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,1.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,1.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ffff64f7850d4268016db8db3d48bf5433db2a926ba71bcf0b17dc4e15f1f223,0.042462,0.0,0.000000,0.625000,0.375000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.125000,0.500000,0.000000,0.250000,0.125000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.500000,0.500000,0.0,0.000000,0.0,0.0,0.000000,0.250000,0.0,0.000000,0.0,0.250000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.500000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
ffffa28cd7ab5d1cbbbfe7b582b1c419270cc0539f3dae5bdfa4cf4b5874c806,0.027102,0.0,0.000000,1.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.500000,0.500000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,1.000000,0.000000,0.0,0.00

## Similarity Function

In [ ]:
def distance(article,customer):
  return 

In [ ]:
from numpy.ma.core import cos
# Can predict about 1000 customers/min
def rec(customer):
  per_articles = articles.copy()
  rec_scores = []
  per_articles['score'] = (1 - np.cos(articles - customer)).sum(axis=1)
  
  return per_articles.sort_values('score').head(5).index.tolist()

In [ ]:
customers

,price,Accessories,Garment Full body,Garment Lower body,Garment Upper body,Nightwear,Shoes,Socks & Tights,Swimwear,Underwear,Unknown,Beige,Black,Blue,Brown,Green,Grey,Khaki green,Lilac Purple,Mole,Orange,Pink,Red,Turquoise,Unknown,White,Yellow,undefined,Baby/Children,Divided,Ladieswear,Menswear,Sport,Denim Men,Divided Accessories,Divided Basics,Divided Collection,Divided Complements Other,Divided Selected,H&M+,Ladies Denim,Ladies H&M Sport,Mama,Men Suits & Tailoring,Men Underwear,Womens Big accessories,Womens Casual,Womens Everyday Basics,Womens Everyday Collection,Womens Jackets,Womens Lingerie,"Womens Nightwear, Socks & Tigh",Womens Premium,Womens Shoes,Womens Small accessories,"Womens Swimwear, beachwear",Womens Tailoring,Young Boy,Young Girl
customer_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0000423b00ade91418cceaf3b26c6af3dd342b51fd051eec9c12fb36984420fa,0.021169,0.0,0.000000,0.000000,1.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.500000,0.500000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,1.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.500000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.500000,0.0,0.0
00007d2de826758b65a93dd24ce629ed66842531df6699338c5570910a014cc2,0.018345,0.0,0.083333,0.166667,0.500000,0.0,0.0,0.0,0.000000,0.250000,0.0,0.000000,0.500000,0.000000,0.083333,0.083333,0.000000,0.0,0.0,0.0,0.000000,0.166667,0.000000,0.0,0.0,0.166667,0.0,0.0,0.0,0.500000,0.500000,0.0,0.000000,0.0,0.0,0.166667,0.250000,0.0,0.083333,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.166667,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
00008469a21b50b3d147c97135e25b4201a8c58997f78782a0cc706645e14493,0.016932,0.0,0.000000,0.000000,1.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.500000,0.000000,0.000000,0.000000,0.000000,0.500000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,1.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.5,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.500000,0.0,0.0
00009d946eec3ea54add5ba56d5210ea898def4b46c68570cf0096d962cacc75,0.029837,0.0,0.000000,0.636364,0.363636,0.0,0.0,0.0,0.000000,0.000000,0.0,0.090909,0.545455,0.090909,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.090909,0.000000,0.000000,0.0,0.0,0.181818,0.0,0.0,0.0,0.181818,0.636364,0.0,0.181818,0.0,0.0,0.090909,0.090909,0.0,0.000000,0.0,0.000000,0.181818,0.0,0.0,0.0,0.0,0.0,0.0,0.636364,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
0000b2f1829e23b24feec422ef13df3ccedaedc85368e6664d04ca30b2f8daff,0.015237,0.0,0.000000,0.000000,1.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,1.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,1.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ffff64f7850d4268016db8db3d48bf5433db2a926ba71bcf0b17dc4e15f1f223,0.042462,0.0,0.000000,0.625000,0.375000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.125000,0.500000,0.000000,0.250000,0.125000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.500000,0.500000,0.0,0.000000,0.0,0.0,0.000000,0.250000,0.0,0.000000,0.0,0.250000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.500000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
ffffa28cd7ab5d1cbbbfe7b582b1c419270cc0539f3dae5bdfa4cf4b5874c806,0.027102,0.0,0.000000,1.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.500000,0.500000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,1.000000,0.000000,0.0,0.00

In [ ]:
top10 = dat_train.groupby('article_id').count().head(10).index.tolist()



In [ ]:
dat_test = dat_test[dat_test['customer_id'].isin(customers.index)]

In [ ]:
dat_test = dat_test[['customer_id','article_id']].reset_index()

In [ ]:
len(dat_test)

445253

In [ ]:
preds = []
t10 = []
for i in range(len(tes)):
  preds.append(rec(customers.loc[tes['customer_id'].iloc[i]]))

tes['recs_con'] = preds

In [ ]:
tes

,customer_id,purchases,cf_recs,recs_con
0,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,[666448006],"[673677002, 507910001, 372860001, 537116001, 5...","[710899003, 712216001, 615141002, 697054002, 6..."
1,00007d2de826758b65a93dd24ce629ed66842531df6699...,"[671502001, 681376001, 685687001]","[355072002, 615141002, 355569001, 589222001, 3...","[685689001, 244267032, 708352001, 708379004, 6..."
2,00009d946eec3ea54add5ba56d5210ea898def4b46c685...,"[573085010, 636455003, 684210001]","[706016001, 706016002, 539723005, 712587003, 5...","[695545002, 673799003, 562245001, 712587004, 6..."
3,0000f1c71aafe5963c3d195cf273f7bfd50bbf17761c91...,[632982036],"[673677002, 372860001, 537116001, 673396002, 6...","[557248001, 685814001, 685813001, 557247001, 5..."
4,0000f2ea26b7f0a9175f428c8cf7743e9e10e193465ecd...,[562245059],"[615141002, 673396002, 708352001, 692454002, 6...","[448509017, 635957001, 673901001, 677809002, 7..."
...,...,...,...,...
286516,ffff1c80bd2dede87deda612527c3df06d2fb270b85602...,[561445005],"[565379001, 561445005, 565379002, 108775015, 5...","[629420002, 629420006, 610776040, 728162001, 5..."
286517,ffff64f7850d4268016db8db3d48bf5433db2a926ba71b...,"[711440001, 717490001]","[706016002, 706016001, 554450001, 706016003, 5...","[695263001, 524825014, 712587002, 612481002, 5..."
286518,ffffbbf78b6eaac697a8a5dfbfd2bfa8113ee5b403e474...,"[469658009, 693242005, 699075005]","[684209001, 599580020, 709297002, 709300002, 5...","[590928019, 736573002, 687034009, 699423003, 7..."
286519,ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab5...,[624486001],"[562245001, 562245018, 573716012, 562245004, 5...","[613246004, 697054002, 666448006, 667491006, 5..."


In [ ]:
tes.to_csv('combined_recommendations.csv')

In [ ]:
rec_accs = []
top_ten_accs = []
for i in range(len(dat_test)):
  if dat_test.iloc[i]['article_id'] in dat_test.iloc[i]['recs']:
    rec_accs.append(1)
  else:
    rec_accs.append(0)
  if dat_test.iloc[i]['article_id'] in dat_test.iloc[i]['top_ten']:
    top_ten_accs.append(1)
  else:
    top_ten_accs.append(0)

dat_test['rec_acc'] = rec_accs
dat_test['top_ten_acc'] = top_ten_accs

In [ ]:
dat_test[['rec_acc','top_ten_acc']].mean()

rec_acc        0.106959
top_ten_acc    0.020328
dtype: float64

In [ ]:
dat_test

,index,customer_id,article_id,recs,top_ten,rec_acc,top_ten_acc
0,0,0220d00495dd23177b6500686fd276939fe7820a55881c...,448509001,"[572187001, 660308007, 554479001, 705988005, 7...","[108775015, 108775044, 111565001, 111586001, 1...",0,0
1,1,d7e30d964ea54211213c6f8a5b389774ecdba39ea2bb91...,664405002,"[537346026, 634426008, 568601007, 504154027, 5...","[108775015, 108775044, 111565001, 111586001, 1...",0,0
2,2,f1589a0cfc975b2cac01928cb51f9cbd7871e8ff754a0d...,708352002,"[297067002, 179208001, 436261001, 158340001, 1...","[108775015, 108775044, 111565001, 111586001, 1...",0,0
3,3,4d1944d9a1550dbbf7199b662a1f34fd0c730a04ff3d16...,683662006,"[673638001, 316441024, 611024001, 613456001, 5...","[108775015, 108775044, 111565001, 111586001, 1...",0,0
4,4,eaa0a722fb3ddaf968bf1ea648ade78263bbba052e820c...,680262001,"[469137001, 637673005, 572400001, 697980002, 6...","[108775015, 108775044, 111565001, 111586001, 1...",0,0
...,...,...,...,...,...,...,...
445248,445248,434a0b0a4a6f9539b469023127841c79f9c3dd88675ea6...,610776002,"[469137001, 637673005, 697980002, 662888006, 6...","[108775015, 108775044, 111565001, 111586001, 1...",0,0
445249,445249,ba7acb32f3f9c8064944bd231fd978f57e9bf02e9b9a97...,156231001,"[562245058, 656763001, 712587004, 562245001, 5...","[108775015, 108775044, 111565001, 111586001, 1...",0,1
445250,445250,ad9bfacbf6d3c57e82aacf4d7dcd2fcd606cc40eb02d5c...,706016002,"[554450019, 539723007, 554450005, 664133002, 5...","[108775015, 108775044, 111565001, 111586001, 1...",0,0
445251,445251,543e043dd2a8e680880601d94b3a43b084387c41642788...,351484002,"[636938001, 688873001, 661162001, 237347004, 6...","[108775015, 108775044, 111565001, 111586001, 1...",0,0


## INVESTIGATION INTO COMBINED ACCURACY STARTS HERE


In [ ]:
da = pd.read_csv('/content/drive/MyDrive/social-networks-project/combined_recommendations.csv')

In [ ]:
da = da.drop('Unnamed: 0',1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [ ]:
import ast
import numpy as np

In [ ]:
da['cf_recs'] = da['cf_recs'].apply(lambda x: ast.literal_eval(x))

In [ ]:
da['recs_con'] = da['recs_con'].apply(lambda x: ast.literal_eval(x))

In [ ]:
da['combined_rec'] = da['cf_recs'] + da['recs_con']

In [ ]:
da

,Unnamed: 0,customer_id,purchases,cf_recs,recs_con,combined_rec
0,0,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,[666448006],"[673677002, 507910001, 372860001, 537116001, 5...","[710899003, 712216001, 615141002, 697054002, 6...","[673677002, 507910001, 372860001, 537116001, 5..."
1,1,00007d2de826758b65a93dd24ce629ed66842531df6699...,"[671502001, 681376001, 685687001]","[355072002, 615141002, 355569001, 589222001, 3...","[685689001, 244267032, 708352001, 708379004, 6...","[355072002, 615141002, 355569001, 589222001, 3..."
2,2,00009d946eec3ea54add5ba56d5210ea898def4b46c685...,"[573085010, 636455003, 684210001]","[706016001, 706016002, 539723005, 712587003, 5...","[695545002, 673799003, 562245001, 712587004, 6...","[706016001, 706016002, 539723005, 712587003, 5..."
3,3,0000f1c71aafe5963c3d195cf273f7bfd50bbf17761c91...,[632982036],"[673677002, 372860001, 537116001, 673396002, 6...","[557248001, 685814001, 685813001, 557247001, 5...","[673677002, 372860001, 537116001, 673396002, 6..."
4,4,0000f2ea26b7f0a9175f428c8cf7743e9e10e193465ecd...,[562245059],"[615141002, 673396002, 708352001, 692454002, 6...","[448509017, 635957001, 673901001, 677809002, 7...","[615141002, 673396002, 708352001, 692454002, 6..."
...,...,...,...,...,...,...
286516,286516,ffff1c80bd2dede87deda612527c3df06d2fb270b85602...,[561445005],"[565379001, 561445005, 565379002, 108775015, 5...","[629420002, 629420006, 610776040, 728162001, 5...","[565379001, 561445005, 565379002, 108775015, 5..."
286517,286517,ffff64f7850d4268016db8db3d48bf5433db2a926ba71b...,"[711440001, 717490001]","[706016002, 706016001, 554450001, 706016003, 5...","[695263001, 524825014, 712587002, 612481002, 5...","[706016002, 706016001, 554450001, 706016003, 5..."
286518,286518,ffffbbf78b6eaac697a8a5dfbfd2bfa8113ee5b403e474...,"[469658009, 693242005, 699075005]","[684209001, 599580020, 709297002, 709300002, 5...","[590928019, 736573002, 687034009, 699423003, 7...","[684209001, 599580020, 709297002, 709300002, 5..."
286519,286519,ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab5...,[624486001],"[562245001, 562245018, 573716012, 562245004, 5...","[613246004, 697054002, 666448006, 667491006, 5...","[562245001, 562245018, 573716012, 562245004, 5..."


In [ ]:
da['purchases'] = da['purchases'].apply(lambda x: ast.literal_eval(x))

In [ ]:
da['recs_correct'] = da.apply(
    lambda row: len([value for value in row.purchases if value in row.combined_rec]),axis=1
)

In [ ]:
da

,Unnamed: 0,customer_id,purchases,cf_recs,recs_con,combined_rec,recs_correct
0,0,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,[666448006],"[673677002, 507910001, 372860001, 537116001, 5...","[710899003, 712216001, 615141002, 697054002, 6...","[673677002, 507910001, 372860001, 537116001, 5...",0
1,1,00007d2de826758b65a93dd24ce629ed66842531df6699...,"[671502001, 681376001, 685687001]","[355072002, 615141002, 355569001, 589222001, 3...","[685689001, 244267032, 708352001, 708379004, 6...","[355072002, 615141002, 355569001, 589222001, 3...",0
2,2,00009d946eec3ea54add5ba56d5210ea898def4b46c685...,"[573085010, 636455003, 684210001]","[706016001, 706016002, 539723005, 712587003, 5...","[695545002, 673799003, 562245001, 712587004, 6...","[706016001, 706016002, 539723005, 712587003, 5...",0
3,3,0000f1c71aafe5963c3d195cf273f7bfd50bbf17761c91...,[632982036],"[673677002, 372860001, 537116001, 673396002, 6...","[557248001, 685814001, 685813001, 557247001, 5...","[673677002, 372860001, 537116001, 673396002, 6...",0
4,4,0000f2ea26b7f0a9175f428c8cf7743e9e10e193465ecd...,[562245059],"[615141002, 673396002, 708352001, 692454002, 6...","[448509017, 635957001, 673901001, 677809002, 7...","[615141002, 673396002, 708352001, 692454002, 6...",0
...,...,...,...,...,...,...,...
286516,286516,ffff1c80bd2dede87deda612527c3df06d2fb270b85602...,[561445005],"[565379001, 561445005, 565379002, 108775015, 5...","[629420002, 629420006, 610776040, 728162001, 5...","[565379001, 561445005, 565379002, 108775015, 5...",1
286517,286517,ffff64f7850d4268016db8db3d48bf5433db2a926ba71b...,"[711440001, 717490001]","[706016002, 706016001, 554450001, 706016003, 5...","[695263001, 524825014, 712587002, 612481002, 5...","[706016002, 706016001, 554450001, 706016003, 5...",0
286518,286518,ffffbbf78b6eaac697a8a5dfbfd2bfa8113ee5b403e474...,"[469658009, 693242005, 699075005]","[684209001, 599580020, 709297002, 709300002, 5...","[590928019, 736573002, 687034009, 699423003, 7...","[684209001, 599580020, 709297002, 709300002, 5...",0
286519,286519,ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab5...,[624486001],"[562245001, 562245018, 573716012, 562245004, 5...","[613246004, 697054002, 666448006, 667491006, 5...","[562245001, 562245018, 573716012, 562245004, 5...",0


In [ ]:
da['recs_correct'].mean()

0.2630243507456696

In [ ]:
np.where(da['recs_correct'] > 0,1,0).mean()

0.24584585423057995